**Classification de la question**

Lorsque l'agent reçoit une question, il devra décider si la question est réellement liée au domaine métier ou non. Si oui et si les données sont en plus regroupées par thématiques, une deuxième décision est à prendre : sur laquelle de ces thématiques porte la question.

Si c'est une question métier, le chatbot retournera une réponse pertinente selon sa stratégie ; si non, il déclenchera la composante conversationnelle, qui produira une réponse originale.

Il faut donc mettre en place une stratégie pour la prise de ces décisions et pour la sélection de la réponse.

Quelle que soit l'approche il faudra d'abord :

    prétraiter la base de données (à faire une seule fois et à stocker). Attention, si on vectorise le corpus il faudra garder le vectoriseur (l'enregistrer comme pickle) pour appliquer ensuite le même vectoriseur à la question ;
    prétraiter la question (en temps réel).




# 1. Prétraitement textuel de la base de données

**Lecture du fichier**

In [1]:
import pandas as pd

In [2]:
df_faq = pd.read_pickle('faq_centerPark.pkl')
df_faq.head()

,question,reponse,theme
0,Quels sont les jours d’arrivée ?,Il est possible d’arriver n’importe quel jour ...,Préparer mon séjour
1,Comment évaluer le confort de mon domaine et d...,Le classement par « birdies » évalue l’offre C...,Préparer mon séjour
2,Quels sont les services et activités compris d...,"En réservant votre hébergement, vous bénéficie...",Préparer mon séjour
3,Comment réserver mes activités ?,"Lors de la réservation de votre hébergement, v...",Préparer mon séjour
4,Où trouver le plan du domaine ?,"Sur la page d'accueil de notre site, cliquez s...",Préparer mon séjour


Notre jeu de données est composé de :
- 54 lignes
- 3 colonnes
- 5 thèmes différents

On pose :
- 1 = Préparer mon séjour
- 2 = Réserver et payer
- 3 = Gérer ma réservation
- 4 = Mon séjour
- 5 = Assurances

In [3]:
dic_code_theme = {"Préparer mon séjour": 1,
                  "Réserver et payer": 2,
                  "Gérer ma réservation": 3,
                  "Mon séjour": 4,
                  "Assurances": 5}
dic_decode_theme = {val: key for key, val in dic_code_theme.items()}

In [4]:
df_faq["theme"].replace(dic_code_theme, inplace=True)

In [5]:
df_faq.head()

,question,reponse,theme
0,Quels sont les jours d’arrivée ?,Il est possible d’arriver n’importe quel jour ...,1
1,Comment évaluer le confort de mon domaine et d...,Le classement par « birdies » évalue l’offre C...,1
2,Quels sont les services et activités compris d...,"En réservant votre hébergement, vous bénéficie...",1
3,Comment réserver mes activités ?,"Lors de la réservation de votre hébergement, v...",1
4,Où trouver le plan du domaine ?,"Sur la page d'accueil de notre site, cliquez s...",1


In [6]:
df_question = df_faq[['question', 'theme']]
df_question.rename(columns={'question': 'texte', 'theme': 'theme'}, inplace=True)
df_reponse = df_faq[['reponse', 'theme']]
df_reponse.rename(columns={'reponse': 'texte', 'theme': 'theme'}, inplace=True)

C:\Users\clair\Anaconda3\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [7]:
df_concat = pd.concat([df_question, df_reponse])

In [8]:
df_concat.head(20)

,texte,theme
0,Quels sont les jours d’arrivée ?,1
1,Comment évaluer le confort de mon domaine et d...,1
2,Quels sont les services et activités compris d...,1
3,Comment réserver mes activités ?,1
4,Où trouver le plan du domaine ?,1
5,Puis-je venir avec mon animal domestique ?,1
6,Peut-on accéder au domaine à la journée (sans ...,1
7,Comment réserver un logement adapté aux person...,1
8,Comment recevoir la brochure ?,1
9,Est-il possible d'acheter des billets pour les...,1


In [9]:
# Sauvegarde
df_concat.to_pickle('df_concat.pkl')

# A. Définition de fonctions pour le nettoyage du texte des tweets

In [10]:
import os
import numpy as np
import re
from collections import defaultdict, Counter

# pour le nettoyage du texte
import nltk
import string
import spacy
from nltk.tokenize import word_tokenize, TweetTokenizer
from nltk.stem import SnowballStemmer

# pour la classification
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [11]:
import unidecode

In [12]:
df_concat = pd.read_pickle('df_concat.pkl')
X_train, X_test, y_train, y_test = train_test_split(df_concat['texte'],
                                                    df_concat['theme'],
                                                    train_size=0.7,
                                                    random_state=5)

In [13]:
stemmer = SnowballStemmer('french')
tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True) # tokenizer for tweet
nlp = spacy.load('fr_core_news_sm')

sw = nltk.corpus.stopwords.words('french')
sw += ['être', 'avoir']
sw.sort()

def lemmatise_text(text):
    lst_lematised = [token.lemma_ for token in nlp(text)] 
    return ' '.join(lst_lematised).lower()


def stem_text(text):
    lst_stemmerised = [stemmer.stem(token) for token in tokenizer.tokenize(text)]    
    return ' '.join(lst_stemmerised)


def replace_words_with_pos_tag(text):
    lst_tags = [token.pos_ for token in nlp(text)]    
    return ' '.join(lst_tags)


def ner(text): #entites nommees
    dico_remplacement = {entite_nommee.text : entite_nommee.label_ for entite_nommee in nlp(text).ents}
    for entite_nommee, remplacement in dico_remplacement.items():
        text = text.replace(entite_nommee, remplacement)
    return text


def substitute_punctuation(text):
    return ' '.join(text.replace("'", ' ').translate(str.maketrans('', '', string.punctuation)).split())


def substitute_number(text, url_replacement=''):
    return re.sub(r"\d", url_replacement, text)


def supp(text):
    return text.replace("«", "").replace("’", "").replace("•", "").replace("®", "")


def supprime_accent(txt):
    return unidecode.unidecode(txt)

# B. Nettoyage du texte

### 1. Nettoyage 

On combine quelques fonctions définies en partie A.

In [14]:
X_train_clean = (X_train.apply(lemmatise_text)
                        .apply(stem_text)
                        .apply(substitute_punctuation)
#                         .apply(supprime_accent)
#                         .apply(substitute_number)
#                         .apply(supp)
                )
X_test_clean = (X_test.apply(lemmatise_text)
                       .apply(stem_text)
                       .apply(substitute_punctuation)
#                        .apply(supprime_accent)
#                        .apply(substitute_number)
#                        .apply(supp)
                )


### 2. Test de différents modèles

Nous allons tester différents modèles sur le texte des tweets qui ont été nettoyés dans la partie 1.

#### a. Les différents vectoriseurs

In [15]:
#vectoriseur numérique discret

vect_count = CountVectorizer(binary=False)
vect_count.fit(X_train_clean)
X_train_clean_vectorized_count = vect_count.transform(X_train_clean)
X_test_clean_vectorized_count = vect_count.transform(X_test_clean)

In [16]:
#vectoriseur binaire

bin_count = CountVectorizer(binary=True)
bin_count.fit(X_train_clean)
X_train_clean_vectorized_bin = vect_count.transform(X_train_clean)
X_test_clean_vectorized_bin = vect_count.transform(X_test_clean)

In [17]:
#vectoriseur numérique continu : TF-IDF

vect_tfidf = TfidfVectorizer(stop_words=sw)
vect_tfidf.fit(X_train_clean)
X_train_clean_vectorized_tfidf = vect_tfidf.transform(X_train_clean)
X_test_clean_vectorized_tfidf = vect_tfidf.transform(X_test_clean) 

#### b. Les différents modèles

Nous entraînerons des modèles de classification appartenant à quelques familles d'algorithmes d'apprentissage automatique classique. L'objectif est de comparer non seulement les performances des différentes méthodes entre elles, mais aussi la performance d'une même méthode sur des représentations différentes du texte.

##### b.1 DummyClassifier

In [42]:
from sklearn.dummy import DummyClassifier

In [105]:
random_uniform = DummyClassifier(strategy='uniform').fit(X_train_clean_vectorized_count, y_train)
predictions_valid = random_uniform.predict(X_test_clean_vectorized_count)
accuracy_score(y_test, predictions_valid)

0.24242424242424243

In [106]:
random_uniform = DummyClassifier(strategy='most_frequent').fit(X_train_clean_vectorized_bin, y_train)
predictions_valid = random_uniform.predict(X_test_clean_vectorized_bin)
accuracy_score(y_test, predictions_valid)

0.24242424242424243

In [107]:
random_prop_class = DummyClassifier(strategy='most_frequent').fit(X_train_clean_vectorized_tfidf, y_train)
pred = random_prop_class.predict(X_test_clean_vectorized_tfidf)
accuracy_score(y_test, pred)

0.24242424242424243

##### b.2 Classifieur naïf bayesien

In [46]:
from sklearn.naive_bayes import MultinomialNB

In [108]:
model_nb = MultinomialNB().fit(X_train_clean_vectorized_count, y_train)
predictions_valid = model_nb.predict(X_test_clean_vectorized_count)
accuracy_score(y_test, predictions_valid)

0.6363636363636364

In [109]:
model_nb = MultinomialNB().fit(X_train_clean_vectorized_bin, y_train)
predictions_valid = model_nb.predict(X_test_clean_vectorized_bin)
accuracy_score(y_test, predictions_valid)

0.6363636363636364

In [110]:
model_nb = MultinomialNB().fit(X_train_clean_vectorized_tfidf, y_train)
predictions_valid = model_nb.predict(X_test_clean_vectorized_tfidf)
accuracy_score(y_test, predictions_valid)

0.5454545454545454

#### b.3 Complement NB

In [50]:
from sklearn.naive_bayes import ComplementNB

In [111]:
model_comp = ComplementNB().fit(X_train_clean_vectorized_count, y_train)
predictions_valid = model_comp.predict(X_test_clean_vectorized_count)
accuracy_score(y_test, predictions_valid)

0.6060606060606061

In [112]:
model_comp = ComplementNB().fit(X_train_clean_vectorized_bin, y_train)
predictions_valid = model_comp.predict(X_test_clean_vectorized_bin)
accuracy_score(y_test, predictions_valid)

0.6060606060606061

In [113]:
model_comp = ComplementNB().fit(X_train_clean_vectorized_tfidf, y_train)
predictions_valid = model_comp.predict(X_test_clean_vectorized_tfidf)
accuracy_score(y_test, predictions_valid)

0.6666666666666666

#### b.4 BernoulliNB

In [54]:
from sklearn.naive_bayes import BernoulliNB

In [114]:
model_bern = BernoulliNB().fit(X_train_clean_vectorized_count, y_train)
predictions_valid = model_bern.predict(X_test_clean_vectorized_count)
accuracy_score(y_test, predictions_valid)

0.36363636363636365

In [115]:
model_bern = BernoulliNB().fit(X_train_clean_vectorized_bin, y_train)
predictions_valid = model_bern.predict(X_test_clean_vectorized_bin)
accuracy_score(y_test, predictions_valid)

0.36363636363636365

In [116]:
model_bern = BernoulliNB().fit(X_train_clean_vectorized_tfidf, y_train)
predictions_valid = model_bern.predict(X_test_clean_vectorized_tfidf)
accuracy_score(y_test, predictions_valid)

0.3333333333333333

#### b.5 RandomForestClassifier/LinearSVC

In [117]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
models = [
    RandomForestClassifier(n_estimators=500, max_depth=100, random_state=0),
    LinearSVC()
]

dic_acc = {}

for model in models:
    model_name = model.__class__.__name__
    predictions_valid = model.fit(X_train_clean_vectorized_count, y_train).predict(X_test_clean_vectorized_count)
    dic_acc[model_name] = accuracy_score(y_test, predictions_valid)

print(dic_acc)

C:\Users\enora\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


{'RandomForestClassifier': 0.45454545454545453, 'LinearSVC': 0.36363636363636365}


#### b.6 KNeighborsClassifier

In [58]:
from sklearn.neighbors import KNeighborsClassifier

In [119]:
model_knn = KNeighborsClassifier(4).fit(X_train_clean_vectorized_count, y_train)
predictions_valid = model_knn.predict(X_test_clean_vectorized_count)
accuracy_score(y_test, predictions_valid)

0.2727272727272727

In [120]:
model_knn = KNeighborsClassifier(4).fit(X_train_clean_vectorized_bin, y_train)
predictions_valid = model_knn.predict(X_test_clean_vectorized_bin)
accuracy_score(y_test, predictions_valid)

0.2727272727272727

In [124]:
model_knn = KNeighborsClassifier(5).fit(X_train_clean_vectorized_tfidf, y_train)
predictions_valid = model_knn.predict(X_test_clean_vectorized_tfidf)
accuracy_score(y_test, predictions_valid)

0.5757575757575758

#### b.7. SVM

In [62]:
from sklearn.svm import SVC

In [125]:
model_svm = SVC(kernel='linear', C=0.9).fit(X_train_clean_vectorized_count, y_train)
predictions_valid = model_svm.predict(X_test_clean_vectorized_count)
accuracy_score(y_test, predictions_valid)

0.3939393939393939

In [126]:
model_svm = SVC(kernel='linear', C=0.1).fit(X_train_clean_vectorized_bin, y_train)
predictions_valid = model_svm.predict(X_test_clean_vectorized_bin)
accuracy_score(y_test, predictions_valid)

0.48484848484848486

In [127]:
model_svm = SVC(kernel='linear', C=0.1).fit(X_train_clean_vectorized_tfidf, y_train)
predictions_valid = model_svm.predict(X_test_clean_vectorized_tfidf)
accuracy_score(y_test, predictions_valid)

0.24242424242424243

#### b.8 LSTM

In [131]:
from keras.models import Sequential
from keras.layers import LSTM, Embedding, Dense, SpatialDropout1D
from keras.callbacks import EarlyStopping

In [143]:
y_train_multi_class = pd.get_dummies(y_train)
y_test_multi_class = pd.get_dummies(y_test)

In [135]:
X_train.shape

(75,)

In [146]:
MAX_NB_WORDS = 50000
EMBEDDING_DIM = 100

model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=(X_train_clean_vectorized_tfidf.shape[1])))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 5
batch_size = 64
history = model.fit(X_train_clean_vectorized_tfidf, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

C:\Users\enora\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 67 samples, validate on 8 samples
Epoch 1/5


InvalidArgumentError:  Received a label value of 5 which is outside the valid range of [0, 1).  Label values: 4 2 4 2 1 5 2 2 4 5 5 5 1 5 1 4 5 2 1 1 2 4 3 1 1 1 4 3 2 3 4 2 2 3 3 4 3 2 4 2 2 1 2 1 2 2 4 3 2 1 3 3 2 4 1 2 4 2 4 4 1 1 5 2
	 [[node loss_5/dense_6_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits (defined at C:\Users\enora\Anaconda3\lib\site-packages\tensorflow_core\python\framework\ops.py:1751) ]] [Op:__inference_keras_scratch_graph_7477]

Function call stack:
keras_scratch_graph


# NN

In [29]:
X_train_clean_vectorized_tfidf.toarray().reshape(75, 733, 1).shape

(75, 733, 1)

In [27]:
maxlen = 400
from keras.preprocessing import sequence
x_train = sequence.pad_sequences(X_train_clean_vectorized_tfidf, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test_clean_vectorized_tfidf, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Using TensorFlow backend.


TypeError: sparse matrix length is ambiguous; use getnnz() or shape[0]

## Modèle vectoriel

### Création du vectoriseur

On passe au vectoriseur use_idf=False pour qu'il soit binaire (0 = absence du terme, 1 = présence du terme).

In [56]:
vectorizer = TfidfVectorizer(lowercase=True, stop_words=None,
                            ngram_range=(1, 1),
                            use_idf=False, smooth_idf=True, # idf lissé
                            sublinear_tf=False, norm='l2')

### Création de la matrice termes-documents

In [105]:
dtm = vectorizer.fit_transform(df['question'])

# Interrogation du corpus

Nous souhaitons trouver le document du corpus qui est le plus similaire à cette requête:

In [108]:
sw = stopwords.words('french')
sw.append('les') # manque dans la liste, par exemple
vect = vectorizer = TfidfVectorizer(lowercase=True, stop_words=None,
                            ngram_range=(1, 1),
                            use_idf=False, smooth_idf=True, # idf lissé
                            sublinear_tf=False, norm='l2')
dtm = vect.fit_transform(df)

def vectorize_query(query_text):
    query_file = 'query.txt'
    with open(query_file, 'w', encoding="utf-8") as out_f:
        out_f.write(query_text)
    query_vector = vect.transform([query_file])
    os.unlink(query_file)
    return query_vector

# Les salutations

Cette fonction sera utilisé pour le message d'acceuil entré par l'utilisateur et la génération de la réponse correspondante.

In [123]:
#Fonction pour les salutations de départ

salutations_inputs = ("salut", "hey", "coucou", "bonjour")
salutations_responses = ["bonjour et bienvenu.e", "bonjour", "bienvenu.e"]

def generate_greeting_response(greeting):
    for token in greeting.split():
        if token.lower() in greeting_inputs:
            return random.choice(greeting_responses)

Nous allons créer une fonction qui prend en entrée l'utilisateur, trouve la similitude en cosinus de 
l'entrée utilisateur et la compare avec les phrases du corpus.

Source : https://stackabuse.com/python-for-nlp-creating-a-rule-based-chatbot/

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

sw = stopwords.words('french')
sw.append('les') # manque dans la liste, par exemple
vect = vectorizer = TfidfVectorizer(lowercase=True, stop_words=sw,
                            ngram_range=(1, 1),
                            use_idf=False, smooth_idf=True, # idf lissé
                            sublinear_tf=False, norm='l2')
dtm = vect.fit_transform(df)

#Vectorisation de l'input utilisateur
def vectorize_query(query_text):
    query_file = 'query.txt'
    with open(query_file, 'w', encoding="utf-8") as out_f:
        out_f.write(query_text) #ecrire l'input utilisateur dans un fichier texte
    query_vector = vect.transform([query_file])
    os.unlink(query_file)
    return query_vector

query = ["test.txt"]
query_vector = vect.transform(query)
#query_vector = vectorize_query(query)
query_corpus_sim = np.squeeze(cosine_similarity(dtm, query_vector))
idx_most_sim = np.argmax(query_corpus_sim)
df[idx_most_sim]
print(df[idx_most_sim])

def get_best_doc(query_text):
    query_vector = vectorize_query(query_text)
    query_corpus_sim = np.squeeze(cosine_similarity(dtm, query_vector))
    doc_id = np.argmax(query_corpus_sim)
    doc_path = df[doc_id] # contient le répertoire parent
    return doc_path

def print_result(query_text):
    doc_path = get_best_doc(query_text)
    doc_filename = os.path.split(doc_path)[-1] # sans le répertoire parent
    print(doc_filename) # affiche le nom du fichier
    print('-' * 20)     # affiche une ligne de '-'
    with open(doc_path, 'r') as in_f:
        print(in_f.read(500) + '...') # affiche les premiers 500 caractères du doc

query = 'politique'
print_result(query)